In [1]:
import tensorflow as tf
import numpy as np
import os
import time

## 讀取數據

In [2]:
path_to_file = "./Three Kingdoms.txt"

In [3]:
# 讀取並為 py2 compat 解碼
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本長度是指文本中的字符個數
print ('Length of text: {} characters'.format(len(text)))

Length of text: 621492 characters


In [4]:
# 看一看文本中的前 250 個字符
print(text[:250])

《三國演義》作者：羅貫中

簡介
　　三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。
　　羅貫中（1330年一1400年之間），名本，號湖海散人，明代通俗小說家。他的籍貫一說是太原（今山西），一說是錢塘（今浙江杭州），不可確考。據傳說，羅貫中曾充任過元末農民起義軍張士誠的幕客．除《三國誌通俗演義》外，


In [5]:
# 文本中的非重複字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

4024 unique characters


# 處理文本
## 向量化文本
在訓練之前，我們需要將字符串映射到數字表示值。創建兩個查找表格：一個將字符映射到數字，另一個將數字映射到字符。

In [6]:
# 創建從非重複字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
# 現在，每個字符都有一個整數表示值。請注意，我們將字符映射至索引 0 至 len(unique).

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '.' :   3,
  '0' :   4,
  '1' :   5,
  '2' :   6,
  '3' :   7,
  '4' :   8,
  '5' :   9,
  '6' :  10,
  '7' :  11,
  '8' :  12,
  '9' :  13,
  '?' :  14,
  '[' :  15,
  ']' :  16,
  '—' :  17,
  '…' :  18,
  '□' :  19,
  ...
}


In [8]:
# 顯示文本首 13 個字符的整數映射
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'《三國演義》作者：羅貫中\r' ---- characters mapped to int ---- > [  23   33  595 1986 2703   24  139 2727 4021 2694 3253   47    1]


## 預測任務
給定一個字符或者一個字符序列，下一個最可能出現的字符是什麼？這就是我們訓練模型要執行的任務。輸入進模型的是一個字符序列，我們訓練這個模型來預測輸出 -- 每個時間步（time step）預測下一個字符是什麼。

由於 RNN 是根據前面看到的元素維持內部狀態，那麼，給定此時計算出的所有字符，下一個字符是什麼？

## 創建訓練樣本和目標
接下來，將文本劃分為樣本序列。每個輸入序列包含文本中的 seq_length 個字符。
對於每個輸入序列，其對應的目標包含相同長度的文本，但是向右順移一個字符。
將文本拆分為長度為 seq_length+1 的文本塊。例如，假設 seq_length 為 4 而且文本為 “Hello”， 那麼輸入序列將為 “Hell”，目標序列將為 “ello”。
為此，首先使用 tf.data.Dataset.from_tensor_slices 函數把文本向量轉換為字符索引流。

In [10]:
# 設定每個輸入句子長度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 創建訓練樣本 / 目標
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

《
三
國
演
義


In [11]:
# batch 方法使我們能輕鬆把單個字符轉換為所需長度的序列。
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'《三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬'
'事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。\r\n\u3000\u3000羅貫中（1330年一1400年之間），名本，號湖海散人，明代通俗小說家。他的籍貫一說是太原（今山西），一說是錢塘（今'
'浙江杭州），不可確考。據傳說，羅貫中曾充任過元末農民起義軍張士誠的幕客．除《三國誌通俗演義》外，他還創作有《隋唐志傳》等通俗小說和《趙太祖龍虎風雲會》等戲劇。另外，有相當一部分人認為《水滸傳》後三十回也'
'是其所作。\r\n\r\n目錄\r\n\r\n第001回\u3000宴桃園豪傑三結義\u3000斬黃巾英雄首立功 第002回\u3000張翼德怒鞭督郵\u3000何國舅謀誅宦豎 \r\n第003回\u3000議溫明董卓叱丁原\u3000饋金珠李肅說呂布 第004回\u3000廢漢帝陳留踐位\u3000'
'謀董賊孟德獻刀 \r\n第005回\u3000發矯詔諸鎮應曹公\u3000破關兵三英戰呂布 第006回\u3000焚金闕董卓行兇\u3000匿玉璽孫堅背約 \r\n第007回\u3000袁紹磐河戰公孫\u3000孫堅跨江擊劉表 第008回\u3000王司徒巧使連環計\u3000董太師大鬧鳳'


In [12]:
# 對於每個序列，使用 map 方法先複製再順移，以創建輸入文本和目標文本。 map 方法可以將一個簡單的函數應用到每一個批次 （batch）。

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
# 打印第一批樣本的輸入與目標值：
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '《三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括'
Target data: '三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬'


這些向量的每個索引均作為一個時間步來處理。作為時間步 0 的輸入，模型接收到 “F” 的索引，並嘗試預測 “i” 的索引為下一個字符。在下一個時間步，模型執行相同的操作，但是 RNN 不僅考慮當前的輸入字符，還會考慮上一步的信息。

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 23 ('《')
  expected output: 33 ('三')
Step    1
  input: 33 ('三')
  expected output: 595 ('國')
Step    2
  input: 595 ('國')
  expected output: 1986 ('演')
Step    3
  input: 1986 ('演')
  expected output: 2703 ('義')
Step    4
  input: 2703 ('義')
  expected output: 24 ('》')


## 創建訓練批次
前面我們使用 tf.data 將文本拆分為可管理的序列。但是在把這些數據輸送至模型之前，我們需要將數據重新排列 （shuffle） 並打包為批次。

In [15]:
# 批大小
BATCH_SIZE = 64

# 設定緩衝區大小，以重新排列數據集
# （TF 數據被設計為可以處理可能是無限的序列，
# 所以它不會試圖在內存中重新排列整個序列。相反，
# 它維持一個緩衝區，在緩衝區重新排列元素。 ） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## 創建模型
使用 tf.keras.Sequential 定義模型。在這個簡單的例子中，我們使用了三個層來定義模型：

* tf.keras.layers.Embedding：輸入層。一個可訓練的對照表，它會將每個字符的數字映射到一個 embedding_dim 維度的向量。
* tf.keras.layers.GRU：一種 RNN 類型，其大小由 units=rnn_units 指定（這裡你也可以使用一個 LSTM 層）。
* tf.keras.layers.Dense：輸出層，帶有 vocab_size 個輸出。

In [16]:
# 詞集的長度
vocab_size = len(vocab)

# 嵌入的維度
embedding_dim = 256

# RNN 的單元數量
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 4024) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           1030144   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 4024)          4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


In [21]:

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([  13,  660, 3331, 1989, 1363, 1578,  164,  423, 3799,  194,   28,
        543, 2448, 1722,  760, 3608, 1343,  661, 2019,  909, 2132, 2798,
       2172,  282, 1062, 2899,  880, 1001, 3903, 3199, 1102,  872, 1672,
        668, 2538,  207,  215,  742,  186,  256, 1050,   65, 1745,  222,
       2355,  359, 1803,  620, 1749, 2971, 1622, 1471, 3151, 2037,  444,
       2104, 1974, 1073,  178, 2170,  217, 3965,  391, 1564, 2846, 1393,
       2311, 3677, 3383, 3656, 1365, 3195, 3739, 1051,  457, 3831, 3481,
       2212, 3202, 2210, 2809,  863, 3801, 2444, 3249, 3858, 1261,  626,
       4019,   66, 2804, 1289, 2361,   84, 3128, 1734, 1682, 1798, 1589,
       2559], dtype=int64)

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '瑞氣非凡。忽見一小童來迎曰：「來者莫非陳孝起乎？」震大驚曰：「仙童如何知我姓字！」童子曰：「吾師昨者有言：今日必有皇帝詔命至；使者必是陳孝起。」震曰：「真神仙也！人言信不誣矣！」遂與小童同入仙莊，拜見'

Next Char Predictions: 
 '9墮踞漣控曷俗厲頭偃』喟秬槽媯鍾捷墳濘崩狀腴玠冬徊莫履廒高謳怏屋梟壕篷傅傯姻候兜彭乾檻像督勒毀垣櫟蘊枉敢誣灌叫爻滕御倉玉傳鸚區暨舸搭盂陞輯闕掩謝霄彰合餅邇璋譎璆膠尪頷租貧馬手基－亂膊抵瞋亦詠樸棍段服粲'


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 4024)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.300023


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
# 檢查點保存至的目錄
checkpoint_dir = './training_checkpoints_2'

# 檢查點的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [27]:
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS=10

In [29]:

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
96/96 [==============================] - 182s 2s/step - loss: 6.1942
Epoch 2/10
96/96 [==============================] - 187s 2s/step - loss: 5.1036
Epoch 3/10
96/96 [==============================] - 187s 2s/step - loss: 4.6236
Epoch 4/10
96/96 [==============================] - 187s 2s/step - loss: 4.3255
Epoch 5/10
96/96 [==============================] - 191s 2s/step - loss: 4.0898
Epoch 6/10
96/96 [==============================] - 186s 2s/step - loss: 3.8868
Epoch 7/10
96/96 [==============================] - 186s 2s/step - loss: 3.7072
Epoch 8/10
96/96 [==============================] - 186s 2s/step - loss: 3.5386
Epoch 9/10
96/96 [==============================] - 187s 2s/step - loss: 3.3756
Epoch 10/10
96/96 [==============================] - 186s 2s/step - loss: 3.2162


# 生成文本
## 恢復最新的檢查點
為保持此次預測步驟簡單，將批大小設定為 1。
由於 RNN 狀態從時間步傳遞到時間步的方式，模型建立好之後只接受固定的批大小。
若要使用不同的 batch_size 來運行模型，我們需要重建模型並從檢查點中恢復權重。

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_2\\ckpt_10'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            1030144   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 4024)           4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


# 預測循環
下面的代碼塊生成文本：
* 首先設置起始字符串，初始化 RNN 狀態並設置要生成的字符個數。
* 用起始字符串和 RNN 狀態，獲取下一個字符的預測分佈。
* 然後，用分類分佈計算預測字符的索引。把這個預測字符當作模型的下一個輸入。
* 模型返回的 RNN 狀態被輸送回模型。現在，模型有更多上下文可以學習，而非只有一個字符。在預測出下一個字符後，更改過的 RNN 狀態被再次輸送回模型。模型就是這樣，通過不斷從前面預測的字符獲得更多上下文，進行學習。


In [33]:
def generate_text(model, start_string):
    # 評估步驟（用學習過的模型生成文本）

    # 要生成的字符個數
    num_generate = 1000

    # 將起始字符串轉換為數字（向量化）
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 空字符串用於存儲結果
    text_generated = []

    # 低溫度會生成更可預測的文本
    # 較高溫度會生成更令人驚訝的文本
    # 可以通過試驗以找到最好的設定
    temperature = 1.0

    # 這裡批大小為 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 刪除批次的維度
        predictions = tf.squeeze(predictions, 0)

        # 用分類分佈預測模型返回的字符
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 把預測字符和前面的隱藏狀態一起傳遞給模型作為下一個輸入
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
print(generate_text(model, start_string=u"孔明 "))

孔明 廳上涕燭而頂，一面相見。慈將免醒過二將及回　分軍前有夏侯霸趕與玄德。王經先斬首，見天子百姓，口皆沉慚。操問：「姓名，段福否？」允取衣帶招軍拔劍取之，自佩劍曰：「背於天地地上裂戍髮中詭鏈擊之。」
　　
　　卻說孔明長安肯出，魏兵大進。孔明亦引三隊兵出城。堅守董亭，秦霸引大兵望漢中而走。羌兵盡乏。魏兵早回。張昭知其人已悅，水中諸將告魏。董玄觸曰：「天子長安世妻英雄，可以待接應，誓斬於丞相相援。」於是嚴顏從人望李歆、敦奉、曹仁鹿著：長子張闓，字：內途西帳。恪文遠問之。策如有劍者，口私惱之。早晚見權起，兵至八口。肅到卻有靜，果然公變。
　　
　　卻說曹操方才出扮，曹仁已晚，一面人殺來，郭嘉施禮畢，回報曰：」城中西擊一路，名曰「字。」張英勸張溫、陸遜因此動靜，遂令徐圖押運過軍。公孫瓚設朝送令其主，奔上營密。
　　袁紹見昱，並皆擢征，看看袁術寨中：此二人家暴獻平來，就借曹操手下一半人馬挑住，大叫：「我親自代與李傕，將在何意？」允曰：「吾曾晚去取糧草，勿憂陣否？」禕曰：「是周瑜之言，某有丞相，教坊之道以宅也。今掌劉琦，骨肉寇明，幸忍相賀。」於是不開口，操兵大進。
　　甘寧領命出城迎之。諸將保聞母殺出，慢軍問曰：「丞相何用此言？」急令三路放回去救主，敕諭已定。
　　
姓破，肝膽羸嗟怨。君與翼德行疏死，好有異謀；若是朝廷，失外時施，恥怨氣概西，將遍就穎上水吏馬丁奉造木牛斗盤，五十人奪於空中；腰懸髭猊自葆。羽葆以卓，其舄斛自終。慎勿舊還。文帝復長王與犬騰，公知愛貴，必生。卿試難與三年！倘有誤見女子，弟與超，大開一日。其秋天賜畢，烈分三路：一可放視。」
　　
　　取天水，提虎自守江東，循種漫紅，取祁山，一
　　得本洞中。臨關、張二弟等一班自思曰：「徐吉、潘璋料敵兵旗開門，欲待攻於樹下耳。」姜維受計，曰：「低頭先生，不知必藉屈此洞。」有
